# Addition

This coursework consist of two parts. The first part includes adding two decimal numbers then generating a binary number output. That did not work so well during test time.

The second part is a conventional classifier using two decimal numbers then generating a 201 class y_train output. It takes a really long time to train.

## Part 1

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import keras
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

import tensorflow as tf
import numpy as np

def addDS():
    total = []
    src1 =[]
    src2 =[]
    #x1 = np.arange(0,101)
    #x2 = np.arange(0,101)
    x1 = np.random.randint(0,100,size=(1000))
    x2 = np.random.randint(0,100,size=(1000))
    r = np.where(x1==81)
    x1=np.delete(x1,r,axis=0)
    x2=np.delete(x2,r,axis=0)
    for u  in x1:
        for v in x2:
            singleInst = [int(d) for d in str(np.binary_repr(u+v,width=8))]
            total += [singleInst]
            src1 += [u]
            src2 += [v]
    return total,src1,src2

x1 = np.random.randint(0,100,size=(10))
print(x1)

    

total,x1,x2 = addDS()

print(len(x1))

dataIdx = np.random.permutation(len(total))
x1 = np.array(x1)
x2 = np.array(x2)
total = np.array(total)
xtrain = np.hstack((x1[dataIdx,np.newaxis],x2[dataIdx,np.newaxis]))
ytrain = total[dataIdx,:]
xtrain = np.divide(xtrain,100)

print(xtrain[50])
print(np.argmax(ytrain[50]))


print(xtrain.shape)
print(ytrain.shape)

inp = Input(shape=(2,))
x = Dense(1,activation='relu')(inp)
x = Dense(200,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(200,activation='relu')(x)
out = Dense(8,activation = 'sigmoid')(x)

model = Model(inp,out)
model.compile(optimizer='adam',loss='mse',metrics=['acc'])

print(model.metrics_names)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,epsilon=0.01,cooldown=1,
              patience=2, min_lr=0.0001)
K.set_value(model.optimizer.lr, 0.001)
model.fit(xtrain,ytrain,batch_size=512,epochs=5,callbacks=[reduce_lr])


Using TensorFlow backend.


[ 9 38  0 82 31 59 80 31 13 33]
980100
[ 0.37  0.55]
1
(980100, 2)
(980100, 8)
['loss', 'acc']
Epoch 1/5
980100/980100 [==============================] - 19s - loss: 0.1943 - acc: 0.8061    
Epoch 2/5
980100/980100 [==============================] - 16s - loss: 0.1690 - acc: 0.8979    
Epoch 3/5
980100/980100 [==============================] - 15s - loss: 0.1617 - acc: 0.8834    
Epoch 4/5
980100/980100 [==============================] - 13s - loss: 0.1570 - acc: 0.8857    
Epoch 5/5
980100/980100 [==============================] - 14s - loss: 0.1540 - acc: 0.8825    


ValueError: Error when checking : expected input_1 to have shape (None, 2) but got array with shape (2, 1)

In [5]:
out = model.predict(np.array([[81,1]]))

In [12]:
def bool2int(x):
    y = 0
    for i,j in enumerate(x):
        y += j<<i
    return y

print(bool2int(np.round(out[0]).astype(int)))

13


## Part 2

In [1]:
#test 2
from keras.layers import *
%matplotlib inline
import matplotlib.pyplot as plt

import keras
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

import tensorflow as tf
import numpy as np

def addDS():
    total = []
    src1 =[]
    src2 =[]
    #x1 = np.arange(0,101)
    #x2 = np.arange(0,101)
    x1 = np.random.randint(0,100,size=(1000))
    x2 = np.random.randint(0,100,size=(1000))
    r = np.where(x1==81)
    x1=np.delete(x1,r,axis=0)
    x2=np.delete(x2,r,axis=0)
    for u  in x1:
        for v in x2:
            total += [u+v]
            src1 += [u]
            src2 += [v]
    return total,src1,src2
total,x1,x2 = addDS()

dataIdx = np.random.permutation(len(total))
x1 = np.array(x1)
x2 = np.array(x2)
total = np.array(total)
xtrain = np.hstack((x1[dataIdx,np.newaxis],x2[dataIdx,np.newaxis]))
ytrain = total[dataIdx]
xtrain = np.divide(xtrain,100)

print(xtrain[50])
print(ytrain[50])
ytrain = keras.utils.to_categorical(ytrain,201)

Using TensorFlow backend.


[ 0.43  0.3 ]
73


In [12]:
def denseBNActDr(hu,x):
    x = Dense(hu)(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    x = Dropout(0.1)(x)
    return x

inp = Input(shape=(2,))
x = Dense(16)(inp)
x = Activation(activation='relu')(x)
x = Dropout(0.1)(x)
x = denseBNActDr(256,x)
x = denseBNActDr(1024,x)
x = denseBNActDr(2048,x)
out = Dense(201,activation = 'softmax',name='out')(x)

model = Model(inp,[out])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

print(model.metrics_names)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,epsilon=0.01,cooldown=1,
              patience=2, min_lr=0.0001)
y_train = {'out': ytrain}
model.fit(xtrain,y_train,batch_size=512,epochs=5,callbacks=[])

['loss', 'acc']
Epoch 1/5
978121/978121 [==============================] - 207s - loss: 2.6435 - acc: 0.1288   
Epoch 2/5
978121/978121 [==============================] - 201s - loss: 2.3515 - acc: 0.1606   
Epoch 3/5
978121/978121 [==============================] - 204s - loss: 2.2419 - acc: 0.1778   
Epoch 4/5
978121/978121 [==============================] - 205s - loss: 2.1385 - acc: 0.1989   
Epoch 5/5
978121/978121 [==============================] - 205s - loss: 2.0379 - acc: 0.2189   


In [15]:
model.fit(xtrain,ytrain,batch_size=512,epochs=5,callbacks=[],shuffle=True)
out = model.predict(np.array([[81,1]]))
print(np.argmax(out[0]))

Epoch 1/5
978121/978121 [==============================] - 205s - loss: 1.9300 - acc: 0.2451   
Epoch 2/5
978121/978121 [==============================] - 204s - loss: 1.8498 - acc: 0.2671   
Epoch 3/5
978121/978121 [==============================] - 205s - loss: 1.7695 - acc: 0.2894   
Epoch 4/5
978121/978121 [==============================] - 206s - loss: 1.6807 - acc: 0.3188   
Epoch 5/5
978121/978121 [==============================] - 204s - loss: 1.6126 - acc: 0.3405   
195
